In [75]:
import os
import pandas as pd
path = r'C:\STUDY\code\tiki_data'

## Data Preparation tổng quan

Trong phần này sẽ:
- Khởi tạo các DataFrame từ các file CSV đã crawl (`product_details`, `products`, `sellers`, `brand_nan_rows`).
- Thực hiện các bước EDA cơ bản: `head()`, `info()`, `describe()`, kiểm tra giá trị NULL, kiểm tra kiểu dữ liệu.
- Làm sạch dữ liệu cho `product_details_df` theo các rule đã mô tả (brand, seller thiếu, badge_names, bỏ cột categories).
- Chuẩn hoá lại một số kiểu dữ liệu thời gian để tiện phân tích sau này.


In [ ]:
# Load các bảng dữ liệu chính
from IPython.display import display

product_details_path = os.path.join(path, 'product_details_20251109_110442.csv')
products_path = os.path.join(path, 'products_20251109_110441.csv')
sellers_path = os.path.join(path, 'sellers_20251109_110442.csv')

product_details_df = pd.read_csv(product_details_path)
products_df = pd.read_csv(products_path)
sellers_df = pd.read_csv(sellers_path)

print("Kích thước các bảng (rows, cols):")
for name, df in [
    ('product_details_df', product_details_df),
    ('products_df', products_df),
    ('sellers_df', sellers_df),
]:
    print(f"- {name}: {df.shape}")


Kích thước các bảng (rows, cols):
- product_details_df: (26187, 16)
- products_df: (3865, 7)
- sellers_df: (709, 5)
- brand_nan_rows_df: (105, 16)


## Data Preparation cho các bảng lịch sử (rating, sales, price)

Trong phần này sẽ xử lý thêm 3 bảng lịch sử:
- `rating_history_20251109_110442.csv` → `rating_history_df`
- `sales_history_20251109_110441.csv` → `sales_history_df`
- `price_history_20251109_110441.csv` → `price_history_df`

Với mỗi bảng sẽ thực hiện:
- EDA cơ bản: `head()`, `info()`, `describe()`, kiểm tra NULL, kiểm tra kiểu dữ liệu.
- Chuẩn hoá cột thời gian `crawl_timestamp` sang kiểu `datetime`.
- Đảm bảo các cột số (rating, review_count, quantity_sold, price, discount, ...) đang ở đúng kiểu số, sẵn sàng cho phân tích chuỗi thời gian và trực quan hoá sau này.


In [ ]:
# Load các bảng lịch sử rating, sales, price
rating_history_path = os.path.join(path, 'rating_history_20251109_110442.csv')
sales_history_path = os.path.join(path, 'sales_history_20251109_110441.csv')
price_history_path = os.path.join(path, 'price_history_20251109_110441.csv')

rating_history_df = pd.read_csv(rating_history_path)
sales_history_df = pd.read_csv(sales_history_path)
price_history_df = pd.read_csv(price_history_path)

print("Kích thước các bảng lịch sử (rows, cols):")
for name, df in [
    ('rating_history_df', rating_history_df),
    ('sales_history_df', sales_history_df),
    ('price_history_df', price_history_df),
]:
    print(f"- {name}: {df.shape}")


In [ ]:
# EDA cơ bản cho các bảng lịch sử: head(), info(), describe(), NULL, dtypes
from IPython.display import display


def basic_eda(df, name, n_head=5):
    print("=" * 100)
    print(f"DataFrame: {name}")
    print("- Kích thước:", df.shape)
    print("\n- 5 dòng đầu tiên (head):")
    display(df.head(n_head))

    print("\n- Thông tin tổng quan (info):")
    print(df.info())

    print("\n- Thống kê mô tả (describe, include='all'):")
    display(df.describe(include='all').transpose())

    print("\n- Số lượng giá trị NULL theo cột:")
    print(df.isnull().sum().sort_values(ascending=False))

    print("\n- Kiểu dữ liệu (dtypes):")
    print(df.dtypes)


for name, df in [
    ('rating_history_df', rating_history_df),
    ('sales_history_df', sales_history_df),
    ('price_history_df', price_history_df),
]:
    basic_eda(df, name)


### Chuẩn hoá kiểu dữ liệu cho các bảng lịch sử

**Vấn đề (nguyên nhân):**
- Cột `crawl_timestamp` ở cả 3 bảng đang ở dạng chuỗi → khó lọc/sort theo thời gian.
- Các cột số như `rating_average`, `review_count`, `quantity_sold`, `all_time_quantity_sold`, `price`, `original_price`, `discount`, `discount_rate` cần đảm bảo đúng kiểu số (float/int) để phân tích thời gian & vẽ biểu đồ.

**Hành động:**
- Chuyển `crawl_timestamp` sang `datetime` cho cả 3 DataFrame.
- Dùng `pd.to_numeric(..., errors='coerce')` cho các cột số quan trọng để đảm bảo kiểu đúng, đồng thời kiểm tra xem có giá trị không parse được (bị chuyển thành NaN) hay không.

**Kết quả mong đợi:**
- `crawl_timestamp` có kiểu `datetime64[ns]`.
- Các cột số là `int64` hoặc `float64`, không còn string lẫn trong cột số.
- Sau bước chuẩn hoá, kiểm tra lại `info()` và `isnull().sum()` để nắm rõ số lượng NULL (nếu có).


In [ ]:
# Chuẩn hoá crawl_timestamp và các cột số cho 3 bảng lịch sử

# 1. Chuẩn hoá cột thời gian
for df, name in [
    (rating_history_df, 'rating_history_df'),
    (sales_history_df, 'sales_history_df'),
    (price_history_df, 'price_history_df'),
]:
    if 'crawl_timestamp' in df.columns:
        df['crawl_timestamp'] = pd.to_datetime(df['crawl_timestamp'])
        print(f"Đã convert crawl_timestamp sang datetime cho {name}")

# 2. Chuẩn hoá các cột số
numeric_cols_rating = ['rating_average', 'review_count']
numeric_cols_sales = ['quantity_sold', 'all_time_quantity_sold']
numeric_cols_price = ['price', 'original_price', 'discount', 'discount_rate']

for cols, df, name in [
    (numeric_cols_rating, rating_history_df, 'rating_history_df'),
    (numeric_cols_sales, sales_history_df, 'sales_history_df'),
    (numeric_cols_price, price_history_df, 'price_history_df'),
]:
    for col in cols:
        if col in df.columns:
            before_non_numeric = df[col].isna().sum()
            df[col] = pd.to_numeric(df[col], errors='coerce')
            after_non_numeric = df[col].isna().sum()
            if after_non_numeric > before_non_numeric:
                print(f"Cột {col} trong {name}: có {after_non_numeric - before_non_numeric} giá trị không parse được (bị chuyển thành NaN)")

print("\nINFO sau khi chuẩn hoá các bảng lịch sử:")
for name, df in [
    ('rating_history_df', rating_history_df),
    ('sales_history_df', sales_history_df),
    ('price_history_df', price_history_df),
]:
    print("=" * 100)
    print(name)
    print(df.info())
    print("\nSố lượng NULL theo cột:")
    print(df.isnull().sum().sort_values(ascending=False))


In [77]:
# EDA cơ bản cho từng DataFrame: head(), info(), describe(), NULL, dtypes

def basic_eda(df, name, n_head=5):
    print("=" * 100)
    print(f"DataFrame: {name}")
    print("- Kích thước:", df.shape)
    print("\n- 5 dòng đầu tiên (head):")
    display(df.head(n_head))

    print("\n- Thông tin tổng quan (info):")
    print(df.info())

    print("\n- Thống kê mô tả (describe, include='all'):")
    display(df.describe(include='all').transpose())

    print("\n- Số lượng giá trị NULL theo cột:")
    print(df.isnull().sum().sort_values(ascending=False))

    print("\n- Kiểu dữ liệu (dtypes):")
    print(df.dtypes)


for name, df in [
    ('product_details_df', product_details_df),
    ('products_df', products_df),
    ('sellers_df', sellers_df),
]:
    basic_eda(df, name)


DataFrame: product_details_df
- Kích thước: (26187, 16)

- 5 dòng đầu tiên (head):


,id,product_id,product_name,category_name,brand,categories,specifications,badges,seller_id,seller_name,seller_total_follower,crawl_timestamp,brand_name,spec_count,badge_count,badge_names
0,8267,54665,Chuột không dây Logitech M185 - Hãng chính hãng,Thiết bị KTS phụ kiện số,"{""id"": 18984, ""name"": ""Logitech"", ""slug"": ""log...","{""id"": 1815, ""name"": ""Thiết Bị Số - Phụ Kiện S...","[{""name"": ""Content"", ""attributes"": [{""code"": ""...","[{""placement"": ""pdp_badge"", ""code"": ""return_po...",1.0,Tiki Trading,511587.0,2025-11-07 11:12:23,Logitech,2,2,","
1,12017,54665,Chuột không dây Logitech M185 - Hãng chính hãng,Thiết bị KTS phụ kiện số,"{""id"": 18984, ""name"": ""Logitech"", ""slug"": ""log...","{""id"": 1815, ""name"": ""Thiết Bị Số - Phụ Kiện S...","[{""name"": ""Content"", ""attributes"": [{""code"": ""...","[{""placement"": ""pdp_badge"", ""code"": ""return_po...",1.0,Tiki Trading,511587.0,2025-11-07 20:38:56,Logitech,2,2,","
2,15767,54665,Chuột không dây Logitech M185 - Hãng chính hãng,Thiết bị KTS phụ kiện số,"{""id"": 18984, ""name"": ""Logitech"", ""slug"": ""log...","{""id"": 1815, ""name"": ""Thiết Bị Số - Phụ Kiện S...","[{""name"": ""Content"", ""attributes"": [{""code"": ""...","[{""placement"": ""pdp_badge"", ""code"": ""return_po...",1.0,Tiki Trading,511587.0,2025-11-08 08:46:01,Logitech,2,2,","
3,19517,54665,Chuột không dây Logitech M185 - Hãng chính hãng,Thiết bị KTS phụ kiện số,"{""id"": 18984, ""name"": ""Logitech"", ""slug"": ""log...","{""id"": 1815, ""name"": ""Thiết Bị Số - Phụ Kiện S...","[{""name"": ""Content"", ""attributes"": [{""code"": ""...","[{""placement"": ""pdp_badge"", ""code"": ""return_po...",1.0,Tiki Trading,511587.0,2025-11-08 18:35:53,Logitech,2,2,","
4,23267,54665,Chuột không dây Logitech M185 - Hãng chính hãng,Thiết bị KTS phụ kiện số,"{""id"": 18984, ""name"": ""Logitech"", ""slug"": ""log...","{""id"": 1815, ""name"": ""Thiết Bị Số - Phụ Kiện S...","[{""name"": ""Content"", ""attributes"": [{""code"": ""...","[{""placement"": ""pdp_badge"", ""code"": ""return_po...",1.0,Tiki Trading,511587.0,2025-11-09 03:58:04,Logitech,2,2,","



- Thông tin tổng quan (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26187 entries, 0 to 26186
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     26187 non-null  int64  
 1   product_id             26187 non-null  int64  
 2   product_name           26187 non-null  object 
 3   category_name          26187 non-null  object 
 4   brand                  26082 non-null  object 
 5   categories             26187 non-null  object 
 6   specifications         26187 non-null  object 
 7   badges                 26187 non-null  object 
 8   seller_id              26180 non-null  float64
 9   seller_name            26180 non-null  object 
 10  seller_total_follower  26180 non-null  float64
 11  crawl_timestamp        26187 non-null  object 
 12  brand_name             26082 non-null  object 
 13  spec_count             26187 non-null  int64  
 14  badge_count            

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,26187.0,NaN,NaN,NaN,13094.0,7559.680086,1.0,6547.5,13094.0,19640.5,26187.0
product_id,26187.0,NaN,NaN,NaN,181058859.274831,99673885.574062,54665.0,82922008.0,204464247.0,275512781.0,278721384.0
product_name,26187,3841,Thắt lưng nam da bò nguyên chất mặt khóa tự độ...,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_name,26187,15,Chăm sóc nhà cửa,1771,NaN,NaN,NaN,NaN,NaN,NaN,NaN
brand,26082,1026,"{""id"": 111461, ""name"": ""OEM"", ""slug"": ""oem""}",3324,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categories,26187,772,"{""id"": 2, ""name"": ""Root"", ""is_leaf"": false}",1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
specifications,26187,4567,"[{""name"": ""Content"", ""attributes"": [{""code"": ""...",367,NaN,NaN,NaN,NaN,NaN,NaN,NaN
badges,26187,7,"[{""placement"": ""pdp_badge"", ""code"": ""freeship_...",17464,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seller_id,26180.0,NaN,NaN,NaN,91636.893545,107592.32954,1.0,915.0,40888.0,173338.0,362463.0
seller_name,26180,709,Tiki Trading,5697,NaN,NaN,NaN,NaN,NaN,NaN,NaN



- Số lượng giá trị NULL theo cột:
brand                    105
brand_name               105
badge_names                8
seller_id                  7
seller_name                7
seller_total_follower      7
product_name               0
id                         0
badges                     0
specifications             0
categories                 0
category_name              0
product_id                 0
crawl_timestamp            0
spec_count                 0
badge_count                0
dtype: int64

- Kiểu dữ liệu (dtypes):
id                         int64
product_id                 int64
product_name              object
category_name             object
brand                     object
categories                object
specifications            object
badges                    object
seller_id                float64
seller_name               object
seller_total_follower    float64
crawl_timestamp           object
brand_name                object
spec_count                 int64


...


- Số lượng giá trị NULL theo cột:
short_description    1
id                   0
name                 0
url_key              0
category_id          0
category_name        0
created_at           0
dtype: int64

- Kiểu dữ liệu (dtypes):
id                    int64
name                 object
short_description    object
url_key              object
category_id           int64
category_name        object
created_at           object
dtype: object
DataFrame: sellers_df
- Kích thước: (709, 5)

- 5 dòng đầu tiên (head):


,seller_id,seller_name,seller_url,seller_total_follower,last_updated
0,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,511587,2025-11-09 03:57:56
1,53660,Nhà sách Fahasa,https://tiki.vn/cua-hang/nha-sach-fahasa,148650,2025-11-09 03:57:57
2,113115,Deli Official Store,https://tiki.vn/cua-hang/deli-official-store,36789,2025-11-09 03:57:58
3,178519,Biti’s Official Store,https://tiki.vn/cua-hang/bitis-official-store,24416,2025-11-09 03:58:46
4,48574,IGA Nội Thất Thông Minh,https://tiki.vn/cua-hang/igea-noi-that-thong-minh,14991,2025-11-09 03:57:59



- Thông tin tổng quan (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   seller_id              709 non-null    int64 
 1   seller_name            709 non-null    object
 2   seller_url             709 non-null    object
 3   seller_total_follower  709 non-null    int64 
 4   last_updated           709 non-null    object
dtypes: int64(2), object(3)
memory usage: 27.8+ KB
None

- Thống kê mô tả (describe, include='all'):


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
seller_id,709.0,NaN,NaN,NaN,134057.214386,113597.751815,1.0,21452.0,113115.0,226206.0,362463.0
seller_name,709,709,LanMart,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seller_url,709,709,https://tiki.vn/cua-hang/lanmart,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seller_total_follower,709.0,NaN,NaN,NaN,2006.409027,20120.823953,0.0,52.0,248.0,843.0,511587.0
last_updated,709,89,2025-11-09 03:58:00,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN



- Số lượng giá trị NULL theo cột:
seller_id                0
seller_name              0
seller_url               0
seller_total_follower    0
last_updated             0
dtype: int64

- Kiểu dữ liệu (dtypes):
seller_id                 int64
seller_name              object
seller_url               object
seller_total_follower     int64
last_updated             object
dtype: object


## Làm sạch brand và brand_name trong product_details_df

**Vấn đề (nguyên nhân):**
- Cột `brand` đang lưu dạng chuỗi JSON (ví dụ: `{"id": 18984, "name": "Logitech", ...}`), trong đó:
  - Với sản phẩm bình thường, `name` chính là tên thương hiệu.
  - Với sách, `name` có thể là `"Book"` và tác giả nằm trong trường `authors` → nếu lấy thẳng `name` sẽ thành `Book` (không đúng brand mong muốn).
- Cột `brand_name` hiện tại có nhiều giá trị trống/NaN, một số trường hợp chưa được fill từ `brand`.
- Yêu cầu nghiệp vụ: chỉ chấp nhận trường hợp **cả `brand` và `brand_name` cùng thiếu**, khi đó sẽ gán `unknown_brand` để có thể xử lý thủ công sau.

**Hành động:**
- Parse chuỗi JSON trong `brand` để lấy ra tên thương hiệu:
  - Nếu `name != 'Book'` → dùng `name`.
  - Nếu `name == 'Book'` và có `authors` → lấy tên tác giả đầu tiên làm brand.
- Tạo cột tạm `brand_from_json` từ `brand`, sau đó kết hợp với `brand_name` hiện có để tạo `clean_brand_name`.
- Với những dòng mà cả `brand` và `brand_name` đều thiếu → gán `clean_brand_name = 'unknown_brand'`.
- Cuối cùng, thay thế lại cột `brand_name` bằng phiên bản đã làm sạch.

**Kết quả mong đợi:**
- `brand_name` phản ánh đúng thương hiệu:
  - Sản phẩm thường: thương hiệu như `Logitech`, `1980Books`, ...
  - Sách: brand là tên tác giả (ví dụ `Fredrik Backman`) thay vì `Book`.
- Không còn dòng nào có cả `brand` và `brand_name` đều rỗng mà không được gán `unknown_brand`. 


In [78]:
# Hàm parse JSON-like và trích xuất brand từ cột brand
import json
import ast


def parse_json_like(text):
    """Try to parse a JSON-like string into Python object, trả về None nếu thất bại."""
    if pd.isna(text):
        return None
    if isinstance(text, (dict, list)):
        return text
    text = str(text)
    if not text.strip():
        return None
    # Thử parse bằng json.loads trước
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Fallback sang ast.literal_eval nếu chuỗi không hoàn toàn đúng JSON
        try:
            return ast.literal_eval(text)
        except (SyntaxError, ValueError):
            return None


def extract_brand_from_brand_column(brand_value):
    """
    Logic trích xuất brand:
    - Nếu brand là dict và name != 'Book' -> trả về name.
    - Nếu name == 'Book' và có authors -> lấy tên author đầu tiên.
    - Ngược lại -> None (để xử lý tiếp bằng brand_name hiện có hoặc gán unknown_brand).
    """
    data = parse_json_like(brand_value)
    if not isinstance(data, dict):
        return None

    name = data.get('name')
    if name == 'Book':
        authors = data.get('authors') or []
        if isinstance(authors, list) and authors:
            first_author = authors[0]
            if isinstance(first_author, dict):
                return first_author.get('name')
        # Không lấy được author thì fallback về 'Book' để xử lý tiếp ở bước sau
        return 'Book'

    return name


# Tạo cột tạm brand_from_json từ cột brand
product_details_df['brand_from_json'] = product_details_df['brand'].apply(extract_brand_from_brand_column)

print("Số dòng có brand_from_json khác NULL:", product_details_df['brand_from_json'].notna().sum())
print("Một vài giá trị brand_from_json mẫu:")
print(product_details_df['brand_from_json'].dropna().head())

# Kết hợp brand_from_json với brand_name hiện tại để tạo clean_brand_name
product_details_df['brand_name_original'] = product_details_df['brand_name']


def choose_clean_brand(row):
    # Ưu tiên brand_from_json nếu có thông tin
    if pd.notna(row['brand_from_json']) and str(row['brand_from_json']).strip() != '':
        return row['brand_from_json']
    # Nếu không có brand_from_json thì dùng brand_name cũ nếu còn giá trị
    if pd.notna(row['brand_name_original']) and str(row['brand_name_original']).strip() != '':
        return row['brand_name_original']
    # Còn lại sẽ được fill thành 'unknown_brand' ở bước sau
    return pd.NA


product_details_df['clean_brand_name'] = product_details_df.apply(choose_clean_brand, axis=1)

print("\nSố dòng trước khi fill unknown_brand mà clean_brand_name vẫn NULL:",
      product_details_df['clean_brand_name'].isna().sum())

# Với các dòng mà cả brand và brand_name đều thiếu -> gán unknown_brand
mask_both_missing = (
    product_details_df['clean_brand_name'].isna() &
    product_details_df['brand'].isna()
)
print("Số dòng có cả brand & clean_brand_name thiếu (sẽ gán unknown_brand):", mask_both_missing.sum())

product_details_df.loc[mask_both_missing, 'clean_brand_name'] = 'unknown_brand'

print("Số dòng còn NULL trong clean_brand_name sau khi fill unknown_brand:",
      product_details_df['clean_brand_name'].isna().sum())

# Thay thế lại cột brand_name bằng clean_brand_name và drop cột tạm
product_details_df.drop(columns=['brand_name'], inplace=True)
product_details_df.rename(columns={'clean_brand_name': 'brand_name'}, inplace=True)

# Có thể giữ hoặc drop cột brand_from_json & brand_name_original, tuỳ nhu cầu phân tích
product_details_df.drop(columns=['brand_from_json', 'brand_name_original'], inplace=True)

print("\nSample brand_name sau khi làm sạch:")
print(product_details_df[['product_id', 'product_name', 'brand', 'brand_name']].head())


Số dòng có brand_from_json khác NULL: 26082
Một vài giá trị brand_from_json mẫu:
0    Logitech
1    Logitech
2    Logitech
3    Logitech
4    Logitech
Name: brand_from_json, dtype: object

Số dòng trước khi fill unknown_brand mà clean_brand_name vẫn NULL: 105
Số dòng có cả brand & clean_brand_name thiếu (sẽ gán unknown_brand): 105
Số dòng còn NULL trong clean_brand_name sau khi fill unknown_brand: 0

Sample brand_name sau khi làm sạch:
   product_id                                     product_name  \
0       54665  Chuột không dây Logitech M185 - Hãng chính hãng   
1       54665  Chuột không dây Logitech M185 - Hãng chính hãng   
2       54665  Chuột không dây Logitech M185 - Hãng chính hãng   
3       54665  Chuột không dây Logitech M185 - Hãng chính hãng   
4       54665  Chuột không dây Logitech M185 - Hãng chính hãng   

                                               brand brand_name  
0  {"id": 18984, "name": "Logitech", "slug": "log...   Logitech  
1  {"id": 18984, "name": "Logit

## Xử lý các sản phẩm thiếu thông tin seller trong product_details_df

**Vấn đề (nguyên nhân):**
- Một số sản phẩm đã hết hàng hoặc không còn được bán → API trả về thiếu toàn bộ thông tin seller.
- Theo quan sát, những dòng này có các cột liên quan seller bị NULL: `seller_id`, `seller_name`, `seller_total_follower`, `badge_names`.
- Các sản phẩm này không còn giá trị phân tích về seller hoặc hành vi bán hàng.

**Hành động:**
- Xác định các dòng có `seller_id` bị NULL (coi đây là tiêu chí nhận diện sản phẩm không còn seller).
- Thống kê số lượng sản phẩm sẽ bị loại bỏ.
- Loại bỏ các dòng đó khỏi `product_details_df`.

**Kết quả mong đợi:**
- `product_details_df` chỉ giữ lại các sản phẩm còn thông tin seller đầy đủ → dữ liệu ổn định hơn cho phân tích seller / badges.
- Số lượng dòng giảm đi đúng bằng số sản phẩm thiếu `seller_id`. 


## Tổng kết Data Preparation cho toàn bộ bảng

Ở phần này tổng hợp nhanh lại trạng thái **sau khi đã làm sạch** của tất cả các bảng dùng cho phân tích:
- `product_details_df`
- `products_df`
- `sellers_df`
- `price_history_df`
- `rating_history_df`
- `sales_history_df`

Mục tiêu:
- Kiểm tra lại `shape`, khoảng thời gian (min/max timestamp) của từng bảng.
- Xác nhận các kiểu dữ liệu (`dtypes`) đã đúng sau cleaning.
- Xem nhanh cột nào còn NULL để lưu ý khi phân tích / join dữ liệu.


In [83]:
# Tổng kết nhanh các bảng sau Data Preparation

summary_tables = [
    ("product_details_df", product_details_df, "crawl_timestamp"),
    ("products_df", products_df, "created_at"),
    ("sellers_df", sellers_df, "last_updated"),
    ("price_history_df", price_history_df, "crawl_timestamp"),
    ("rating_history_df", rating_history_df, "crawl_timestamp"),
    ("sales_history_df", sales_history_df, "crawl_timestamp"),
]

for name, df, time_col in summary_tables:
    print("=" * 120)
    print(f"DataFrame: {name}")
    print("- Shape (rows, cols):", df.shape)

    # Khoảng thời gian nếu có cột thời gian
    if time_col in df.columns:
        time_min = df[time_col].min()
        time_max = df[time_col].max()
        print(f"- {time_col} range: {time_min} -> {time_max}")

    # NULL theo cột (chỉ in những cột còn NULL)
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if not null_counts.empty:
        print("- Các cột còn NULL:")
        print(null_counts.sort_values(ascending=False))
    else:
        print("- Không còn NULL ở bất kỳ cột nào.")

    print("- Dtypes:")
    print(df.dtypes)
    print()


DataFrame: product_details_df
- Shape (rows, cols): (26180, 15)
- crawl_timestamp range: 2025-11-06 18:24:47 -> 2025-11-09 03:59:16
- Các cột còn NULL:
brand    105
dtype: int64
- Dtypes:
id                                int64
product_id                        int64
product_name                     object
category_name                    object
brand                            object
specifications                   object
badges                           object
seller_id                       float64
seller_name                      object
seller_total_follower           float64
crawl_timestamp          datetime64[ns]
spec_count                        int64
badge_count                       int64
badge_names                      object
brand_name                       object
dtype: object

DataFrame: products_df
- Shape (rows, cols): (3865, 7)
- created_at range: 2025-11-06 18:25:13 -> 2025-11-09 03:59:16
- Các cột còn NULL:
short_description    1
dtype: int64
- Dtypes:
id           

In [84]:
# Loại bỏ các sản phẩm thiếu seller trong product_details_df

n_before = product_details_df.shape[0]
missing_seller_mask = product_details_df['seller_id'].isna()

print("Số sản phẩm thiếu seller_id (sẽ bị loại bỏ):", missing_seller_mask.sum())

product_details_df = product_details_df.loc[~missing_seller_mask].reset_index(drop=True)

n_after = product_details_df.shape[0]
print("Kích thước trước khi xoá:", n_before)
print("Kích thước sau khi xoá:", n_after)
print("Số dòng đã xoá:", n_before - n_after)


Số sản phẩm thiếu seller_id (sẽ bị loại bỏ): 0
Kích thước trước khi xoá: 26180
Kích thước sau khi xoá: 26180
Số dòng đã xoá: 0


## Chuẩn hoá badge_names từ cột badges và bỏ cột categories

**Vấn đề (nguyên nhân):**
- Cột `badges` đang lưu chuỗi JSON dạng list, ví dụ:
  - `[{"placement": "pdp_badge", "code": "freeship_xtra", ...}, {"placement": "pdp_badge", "code": "return_policy", ...}, ...]`
- Cột `badge_names` hiện tại chỉ chứa chuỗi `", , "` hoặc để trống → không phản ánh đúng các badge thực tế.
- Cột `categories` lưu JSON về category (id, name, is_leaf) nhưng không cần dùng trực tiếp ở bước phân tích hiện tại.

**Hành động:**
- Parse chuỗi JSON trong `badges` và trích ra danh sách `code` cho từng sản phẩm → gán vào `badge_names` (dạng list các string).
- Cập nhật lại `badge_count` = độ dài của list `badge_names`.
- Bỏ cột `categories` để giảm nhiễu, vì thông tin category chi tiết đã có ở bảng khác (`products_df`) hoặc có thể phân tích sau bằng cách parse riêng.

**Kết quả mong đợi:**
- `badge_names` phản ánh đúng các badge như: `["freeship_xtra", "return_policy", "is_authentic"]`.
- `badge_count` đúng với số phần tử trong `badge_names`.
- DataFrame gọn hơn, loại bỏ cột `categories` chưa dùng tới.


In [85]:
# Chuẩn hoá badge_names và badge_count, drop cột categories


def extract_badge_codes(badges_value):
    data = parse_json_like(badges_value)
    if not isinstance(data, list):
        return []
    codes = []
    for item in data:
        if isinstance(item, dict):
            code = item.get('code')
            if code is not None:
                codes.append(code)
    return codes


# Tạo badge_names mới từ badges
product_details_df['badge_names'] = product_details_df['badges'].apply(extract_badge_codes)

# Cập nhật lại badge_count theo badge_names
product_details_df['badge_count'] = product_details_df['badge_names'].apply(len)

print("Một vài sample badge_names & badge_count sau khi chuẩn hoá:")
print(product_details_df[['product_id', 'badge_names', 'badge_count']].head(10))

# Drop cột categories nếu tồn tại
if 'categories' in product_details_df.columns:
    product_details_df.drop(columns=['categories'], inplace=True)
    print("\nĐã drop cột 'categories' khỏi product_details_df")


Một vài sample badge_names & badge_count sau khi chuẩn hoá:
   product_id                                        badge_names  badge_count
0       54665                      [return_policy, is_authentic]            2
1       54665                      [return_policy, is_authentic]            2
2       54665                      [return_policy, is_authentic]            2
3       54665                      [return_policy, is_authentic]            2
4       54665                      [return_policy, is_authentic]            2
5      299431       [freeship_xtra, return_policy, is_authentic]            3
6      299431       [freeship_xtra, return_policy, is_authentic]            3
7      299431       [freeship_xtra, return_policy, is_authentic]            3
8      299431       [freeship_xtra, return_policy, is_authentic]            3
9      299431  [is_hero, freeship_xtra, return_policy, is_aut...            4


## Chuẩn hoá kiểu dữ liệu thời gian và kiểm tra lại sau cleaning

**Vấn đề (nguyên nhân):**
- Một số cột thời gian đang ở dạng chuỗi (string):
  - `product_details_df.crawl_timestamp`
  - `products_df.created_at`
  - `sellers_df.last_updated`
- Nếu giữ dạng string sẽ khó lọc theo ngày, sort, resample theo thời gian.

**Hành động:**
- Chuyển các cột trên sang kiểu `datetime` bằng `pd.to_datetime`.
- Sau đó in lại `info()`, `isnull().sum()` cho các bảng để xác nhận:
  - Kiểu dữ liệu đã đúng.
  - Số lượng NULL sau các bước cleaning (brand, seller, badges, drop categories) có phù hợp với kỳ vọng.

**Kết quả mong đợi:**
- Các cột thời gian ở đúng kiểu `datetime64[ns]`.
- DataFrame sau cleaning rõ ràng, sẵn sàng cho các bước phân tích/visualization tiếp theo.


In [86]:
# Chuyển các cột thời gian sang datetime và kiểm tra lại info / NULL

# Chuẩn hoá kiểu datetime
product_details_df['crawl_timestamp'] = pd.to_datetime(product_details_df['crawl_timestamp'])
products_df['created_at'] = pd.to_datetime(products_df['created_at'])
sellers_df['last_updated'] = pd.to_datetime(sellers_df['last_updated'])

print("INFO sau khi chuẩn hoá datetime & cleaning:")
for name, df in [
    ('product_details_df', product_details_df),
    ('products_df', products_df),
    ('sellers_df', sellers_df),
]:
    print("=" * 100)
    print(name)
    print(df.info())
    print("\nSố lượng NULL theo cột:")
    print(df.isnull().sum().sort_values(ascending=False))


INFO sau khi chuẩn hoá datetime & cleaning:
product_details_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     26180 non-null  int64         
 1   product_id             26180 non-null  int64         
 2   product_name           26180 non-null  object        
 3   category_name          26180 non-null  object        
 4   brand                  26075 non-null  object        
 5   specifications         26180 non-null  object        
 6   badges                 26180 non-null  object        
 7   seller_id              26180 non-null  float64       
 8   seller_name            26180 non-null  object        
 9   seller_total_follower  26180 non-null  float64       
 10  crawl_timestamp        26180 non-null  datetime64[ns]
 11  spec_count             26180 non-null  int64         
 1

In [96]:
# Xuất các bảng đã cleaning ra file CSV
product_details_df.to_csv("product_details_cleaned.csv", index=False)
products_df.to_csv("products_cleaned.csv", index=False)
sellers_df.to_csv("sellers_cleaned.csv", index=False)
rating_history_df.to_csv("rating_history_cleaned.csv", index=False)
sales_history_df.to_csv("sales_history_cleaned.csv", index=False)
price_history_df.to_csv("price_history_cleaned.csv", index=False)